In [ ]:
import os
import cv2
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks
import numpy as np
import matplotlib.pyplot as plt

# ========== PARAMETER ==========
RAW_FOLDER = '/kaggle/input/data-gemastik-cv-sementara'         # Folder gambar seperti "Saja_16.jpg"
CROPPED_FOLDER = 'cropped_dataset' # Folder hasil crop tangan
IMAGE_SIZE = (1920, 1080)
BATCH_SIZE = 32
EPOCHS = 20

# ========== 1. Mediapie ==========
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

def draw_landmarks_only_pose_and_hands(image, results):
    annotated = image.copy()
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            annotated,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
        )

    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2)
        )

    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2)
        )

    return annotated

def extract_and_save(img_path, output_base):
    filename = os.path.basename(img_path)

    # Lewati file yang tidak mengandung underscore
    if "_" not in filename:
        print(f"❌ Skip (tidak ada underscore): {filename}")
        return False

    class_name = filename.split("_")[0]
    output_dir = os.path.join(output_base, class_name)
    os.makedirs(output_dir, exist_ok=True)

    img = cv2.imread(img_path)
    if img is None:
        print(f"❌ Gagal baca gambar: {img_path}")
        return False

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Jalankan MediaPipe Holistic
    with mp_holistic.Holistic(
        static_image_mode=True,
        model_complexity=0,
        refine_face_landmarks=False
    ) as holistic:
        result = holistic.process(img_rgb)

        if result.pose_landmarks or result.left_hand_landmarks or result.right_hand_landmarks:
            annotated = draw_landmarks_only_pose_and_hands(img, result)
            annotated = cv2.resize(annotated, IMAGE_SIZE)
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, annotated)
            print(f"✅ {filename} → {class_name}")
            return True
        else:
            print(f"❌ Tidak ada landmark terdeteksi: {filename}")
            return False

if __name__ == "__main__":
    print("📦 Memproses gambar dengan MediaPipe...")
    os.makedirs(CROPPED_FOLDER, exist_ok=True)

    for file in os.listdir(RAW_FOLDER):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            full_path = os.path.join(RAW_FOLDER, file)
            extract_and_save(full_path, CROPPED_FOLDER)


# ========== 2. Data Generator ==========
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    CROPPED_FOLDER,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    CROPPED_FOLDER,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

NUM_CLASSES = len(train_gen.class_indices)

# ========== Cek Data ==========
# Ambil satu batch dari generator
train_img_batch, train_label_batch = next(train_gen)
val_img_batch, val_label_batch = next(val_gen)  # Atau val_gen, tergantung nama kamu

# Ambil gambar pertama dari batch
train_img = train_img_batch[0]
train_label_idx = np.argmax(train_label_batch[0])
train_label = list(train_gen.class_indices.keys())[train_label_idx]

val_img = val_img_batch[0]
val_label_idx = np.argmax(val_label_batch[0])
val_label = list(val_gen.class_indices.keys())[val_label_idx]

# Tampilkan gambar train
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(train_img)
plt.title(f"[TRAIN] Label: {train_label}")
plt.axis("off")

# Tampilkan gambar val
plt.subplot(1, 2, 2)
plt.imshow(val_img)
plt.title(f"[val] Label: {val_label}")
plt.axis("off")

plt.show()


# ========== 3. Model MobileNet ==========
base_model = MobileNetV2(input_shape=IMAGE_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ========== 4. Training ==========
print("🚀 Training dimulai...")
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

# ========== 5. Simpan Model ==========
model.save("mobilenet_gesture_model.h5")
print("✅ Model tersimpan sebagai mobilenet_gesture_model.h5")

# ========== 6. Evaluasi ==========
loss, acc = model.evaluate(val_gen)
print(f"🎯 Akurasi Validasi: {acc:.2%}")

# ========== 7. Inference ==========
from tensorflow.keras.models import load_model

# Muat ulang model
model = load_model("mobilenet_gesture_model.h5")

# Pilih salah satu gambar dari folder CROPPED_FOLDER
sample_class = np.random.choice(os.listdir(CROPPED_FOLDER))
sample_img_path = os.path.join(CROPPED_FOLDER, sample_class, np.random.choice(os.listdir(os.path.join(CROPPED_FOLDER, sample_class))))

# Baca dan preprocess
img = cv2.imread(sample_img_path)
img_resized = cv2.resize(img, IMAGE_SIZE)
img_normalized = img_resized / 255.0
img_input = np.expand_dims(img_normalized, axis=0)

# Prediksi
pred = model.predict(img_input)[0]
pred_idx = np.argmax(pred)
pred_label = list(train_gen.class_indices.keys())[pred_idx]

# Tampilkan
plt.figure(figsize=(6, 4))
plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
plt.title(f"Prediksi: {pred_label} ({pred[pred_idx]:.2%})")
plt.axis("off")
plt.show()
